In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import spacy
from langdetect import detect
import numpy as np

# Cargar modelo BERT solo una vez
model = SentenceTransformer('distiluse-base-multilingual-cased')

# Cargar modelos de spaCy para múltiples idiomas
nlp_models = {
    'es': spacy.load('es_core_news_md'),
    'en': spacy.load('en_core_web_md'),
    'fr': spacy.load('fr_core_news_md')
}

def detectar_idioma(texto):
    return detect(texto)

def cargar_modelo_spacy(idioma):
    return nlp_models.get(idioma, nlp_models['en'])

def extraer_frases_clave(texto, nlp):
    doc = nlp(texto)
    oraciones = [sent.text.strip() for sent in doc.sents]
    return oraciones

def seleccionar_frases_relevantes_por_grupo(df_grupo, num_frases=20):
    # Calcular TF-IDF para las frases del grupo
    vectorizador = TfidfVectorizer()
    tfidf_matrix = vectorizador.fit_transform(df_grupo['Frase'].tolist())
    
    # Obtener las puntuaciones TF-IDF medias para cada frase
    relevancias = tfidf_matrix.mean(axis=1).A1
    
    # Ordenar las frases por relevancia y seleccionar las top N
    frases_relevantes_idx = np.argsort(relevancias)[-num_frases:][::-1]
    df_grupo_relevante = df_grupo.iloc[frases_relevantes_idx]
    
    return df_grupo_relevante

def procesar_y_guardar_articulos(input_excel, output_excel):
    # Cargar los artículos del archivo Excel
    df = pd.read_excel(input_excel)
    
    # Crear un diccionario para los modelos ya cargados
    modelos_cargados = {}

    # Lista para guardar los resultados
    resultados = []

    # Procesar cada artículo
    for _, row in df.iterrows():
        texto = row['Texto']
        id_articulo = row['IDArticulo']
        id_grupo = row['IDGrupo']
        
        # Detectar idioma y cargar el modelo spaCy correspondiente (si no está en cache)
        idioma = detectar_idioma(texto)
        if idioma not in modelos_cargados:
            modelos_cargados[idioma] = cargar_modelo_spacy(idioma)
        
        # Extraer frases clave usando el modelo correspondiente
        frases_clave = extraer_frases_clave(texto, modelos_cargados[idioma])

        # Filtrar frases con más de 4 palabras
        frases_clave = [frase for frase in frases_clave if len(frase.split()) > 4]

        # Añadir frases con su ID de artículo y grupo al DataFrame
        for frase in frases_clave:
            resultados.append({'IDGrupo': id_grupo, 'IDArticulo': id_articulo, 'Frase': frase})

    # Convertir la lista de resultados en un DataFrame
    df_frases = pd.DataFrame(resultados)

    # Agrupar por IDGrupo y seleccionar las frases más relevantes para cada grupo
    df_relevantes_por_grupo = df_frases.groupby('IDGrupo').apply(lambda x: seleccionar_frases_relevantes_por_grupo(x)).reset_index(drop=True)

    # Filtrar grupos que tienen más de 20 frases relevantes
    grupos_validos = df_relevantes_por_grupo.groupby('IDGrupo').filter(lambda x: len(x) >= 20)

    # Guardar los resultados en un archivo Excel
    grupos_validos.to_excel(output_excel, index=False)

procesar_y_guardar_articulos('ArticulosAgrupados.xlsx', 'FrasesArticulos.xlsx')